## 32 Renderização com diferentes shader programs


In [2]:
import glm
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL, QtCore
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.PhongShadingShaderProgram import PhongShadingShaderProgram
from cg.shader_programs.FlatShadingShaderProgram import FlatShadingShaderProgram 
from cg.shader_programs.SimpleShaderProgram_v2 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v3 import ModelRenderer
from cg.models.SphereMesh_v2 import SphereMesh

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        # posiciona a câmera na posição z = 3 olhando para o eixo z engativo
        # com isso, todos os objetos da cena estão na frente da câmera
        self.cameraEye   = glm.vec3(0.0, 0.0, 3.0)
        self.cameraCenter = glm.vec3(0.0, 0.0, -1.0)
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0)
        self.viewMatrix = glm.lookAt(self.cameraEye, self.cameraCenter, self.cameraUp)
        
        # cria uma malha esférica
        sphere_mesh = SphereMesh(0.2, 20, 20)
        
        # cria o objeto responsável por carregar os dados para a GPU e renderizá-los
        # cria um objeto de renderização para cada program shader
        self.phongSphereRenderer = ModelRenderer(sphere_mesh.getVertexPositions(),
                                            vertex_indices=sphere_mesh.getVertexIndices(),
                                            vertex_normal=sphere_mesh.getVertexNormals())
    
        self.flatSphereRenderer = ModelRenderer(sphere_mesh.getVertexPositions(),
                                            vertex_indices=sphere_mesh.getVertexIndices(),
                                            vertex_normal=sphere_mesh.getVertexNormals())
        
        self.simpleSphereRenderer = ModelRenderer(sphere_mesh.getVertexPositions(),
                                            vertex_indices=sphere_mesh.getVertexIndices(),
                                            vertex_normal=sphere_mesh.getVertexNormals())
        
        # posição da luz
        self.lightPosition = glm.vec4(0.0, 0, 0.0, 1)
        
        # cria três shader program diferentes
        # cada shader program realiza uma renderização diferente
        self.phongShaderProgram = PhongShadingShaderProgram()
        self.flatShaderProgram = FlatShadingShaderProgram()
        self.simpleShaderProgram = SimpleShaderProgram()
        
        # configura cada shader programa
        self.phongShaderProgram.bind()
        self.phongShaderProgram.useUniformMaterialColor(True) # tem o mesmo funcionamento da função useUniformColor do SimpleShader
        self.phongShaderProgram.setUniformLightPosition(self.viewMatrix * self.lightPosition) #a posição da luz tem que estar no espaço de visão
        self.phongShaderProgram.release()
        
        self.flatShaderProgram.bind()
        self.flatShaderProgram.useUniformMaterialColor(True)# tem o mesmo funcionamento da função useUniformColor do SimpleShader
        self.flatShaderProgram.setUniformLightPosition(self.viewMatrix * self.lightPosition) #a posição da luz tem que estar no espaço de visão
        self.flatShaderProgram.release()
        
        self.simpleShaderProgram.bind()
        self.simpleShaderProgram.useUniformColor(True)
        self.simpleShaderProgram.release()

        # recupera o endereço da variável de entrada de cada shader program
        # configura os dados do modelo para serem os dados de entrada do shader program
        position_loc = self.phongShaderProgram.getVertexPositionLoc()
        normal_loc = self.phongShaderProgram.getVertexNormalLoc()
        self.phongSphereRenderer.setVertexPositionLoc(position_loc)
        self.phongSphereRenderer.setVertexNormalLoc(normal_loc)
        
        position_loc = self.flatShaderProgram.getVertexPositionLoc()
        normal_loc = self.flatShaderProgram.getVertexNormalLoc()
        self.flatSphereRenderer.setVertexPositionLoc(position_loc)
        self.flatSphereRenderer.setVertexNormalLoc(normal_loc)
        
        position_loc = self.simpleShaderProgram.getVertexPositionLoc()
        self.simpleSphereRenderer.setVertexPositionLoc(position_loc)

        # habilita teste de profundidade e culling
        gl.glEnable(gl.GL_DEPTH_TEST);
        gl.glEnable(gl.GL_CULL_FACE)
        
    def paintGL(self):
        
        # configura a cor de background
        gl.glClearColor(1, 1, 1, 1)
        
        # limpa o background com a cor especificada e o buffer de profundidade
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # renderiza a esfera
        self.renderScene()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
    
    def renderScene(self):
        
        # ativa o shader program para usar phong shading
        self.phongShaderProgram.bind()
        
        self.renderPhongSphere(glm.vec3(0.0, 0.0, -0.5), np.array([0.0, 0.0, 0.5, 1.0])) # esfera azul
        self.renderPhongSphere(glm.vec3(-1.2, -0.8, 0.2), np.array([0.0, 0.5, 0.0, 1.0]))# esfera verde
        self.renderPhongSphere(glm.vec3(-0.5, -0.2, 0.6), np.array([0.5, 0.0, 0.0, 1.0]))# esfera vermelha
        
        # desativa o shader program
        self.phongShaderProgram.release()
        
        # ativa o shader program para usar flat shading
        self.flatShaderProgram.bind()
        self.renderFlatSphere(glm.vec3(0.3, -0.2, -0.3), np.array([0.8, 0.8, 0.0, 1.0]))# esfera amarela
        self.renderFlatSphere(glm.vec3(-0.6, 0.7, -0.5), np.array([0.0, 0.5, 0.5, 1.0]))# esfera ciano
        self.renderFlatSphere(glm.vec3(0.5, 0.2, 0.3), np.array([0.5, 0.0, 0.5, 1.0]))  # esfera roxa
        
        # desativa o shader program
        self.flatShaderProgram.release()
        
        # ativa o shader program para renderizar sem iluminação
        self.simpleShaderProgram.bind()
        self.renderSimpleSphere(glm.vec3(0.1, 0.8, -1.2), np.array([0.8, 0.4, 0.0, 1.0])) # esfera laranja
        self.renderSimpleSphere(glm.vec3(0.0, -0.8, 1.2), np.array([0.0, 0.0, 0.0, 1.0]))# esfera preta
        
        # desativa o shader program
        self.simpleShaderProgram.release()
        
    def renderPhongSphere(self, transl, color):
        
        # calcula a matriz de translação da esfera e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), transl)
        self.phongShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        #configura a matriz model-view utilizada no cálculo da iluminação
        self.phongShaderProgram.setUniformModelViewMatrix(self.viewMatrix * model_matrix)
        
        # mudar a cor no shader e renderiza a esfera
        self.phongShaderProgram.setUniformMaterialColor(color)
        self.phongSphereRenderer.render()
        
    def renderFlatSphere(self, transl, color):
        
        # calcula a matriz de translação da esfera e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), transl)
        self.flatShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        #configura a matriz model-view utilizada no cálculo da iluminação
        self.flatShaderProgram.setUniformModelViewMatrix(self.viewMatrix * model_matrix)
        
        # mudar a cor no shader e renderiza a esfera
        self.flatShaderProgram.setUniformMaterialColor(color)
        self.flatSphereRenderer.render()
    
    def renderSimpleSphere(self, transl, color):
        
        # calcula a matriz de translação da esfera e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), transl)
        self.simpleShaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # mudar a cor no shader e renderiza a esfera
        self.simpleShaderProgram.setUniformColor(color)
        self.simpleSphereRenderer.render()
        
    def resizeGL(self, width, height):
        
        gl.glViewport(0, 0, width, height)
        
        aspectRatio = width / height
                
        # define a distância do 'far plane' o suficiente para renderizar todo os objetos da cena
        self.perspectiveMatrix = glm.perspective(glm.radians(60.0), aspectRatio, 0.1, 5.0)

    def keyPressEvent(self, event):
        super(MyWidget, self).keyPressEvent(event)
        
        # passo 
        step = 0.1
        
        # verifica se foi pressionada a tecla de seta 'para cima'
        if event.key() == QtCore.Qt.Key_Up:
            self.lightPosition.y += step

        # verifica se foi pressionada a tecla de seta 'para baixo'
        elif event.key() == QtCore.Qt.Key_Down:
            self.lightPosition.y -= step
        
        # verifica se foi pressionada a tecla de seta 'para esquerda'
        elif event.key() == QtCore.Qt.Key_Left:
            self.lightPosition.x -= step
        
        # verifica se foi pressionada a tecla de seta 'para direita'
        elif event.key() == QtCore.Qt.Key_Right:
            self.lightPosition.x += step
        
        # verifica se foi pressionada a tecla de seta '-'
        elif event.key() == QtCore.Qt.Key_Minus:
            self.lightPosition.z -= step
        
        # verifica se foi pressionada a tecla de seta '+'
        elif event.key() == QtCore.Qt.Key_Plus:
            self.lightPosition.z += step
        
        self.flatShaderProgram.bind()
        self.flatShaderProgram.setUniformLightPosition(self.viewMatrix * self.lightPosition)
        self.flatShaderProgram.release()
        
        self.phongShaderProgram.bind()
        self.phongShaderProgram.setUniformLightPosition(self.viewMatrix * self.lightPosition)
        self.phongShaderProgram.release()
        
def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(800, 800)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [33]:
! jupyter nbconvert --to python 32_Esferas_com_diferentes_shader_programs.ipynb
%run -i 32_Esferas_com_diferentes_shader_programs.py

[NbConvertApp] Converting notebook 32_Esferas_iluminacao.ipynb to python
[NbConvertApp] Writing 10068 bytes to 32_Esferas_iluminacao.py
